In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader

In [24]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)
print("torch.cuda.device_count():", torch.cuda.device_count())

Torch version: 2.10.0.dev20251211+cu130
CUDA available: False
torch.version.cuda: 13.0
torch.cuda.device_count(): 0


In [23]:
print(torch.version.cuda)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using: {device}")

13.0
Using: cpu


In [4]:
transform = transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

In [5]:
dataset0  = datasets.ImageFolder(root='./dataset/',transform=None)
class_names = dataset0.classes
print(class_names)
print(len(class_names))

['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
4


In [7]:
class DataModule(pl.LightningDataModule):
    def __init__(self, transform = transform, batch_size = 32):
        super().__init__()
        self.root_dir = './dataset/'
        self.transform = transform
        self.batch_size = batch_size
    
    def setup(self,stage=None):
        dataset = datasets.ImageFolder(root = self.root_dir,transform=self.transform)
        n_data = len(dataset)
        n_train = int(0.7 * n_data)
        n_test = n_data - n_train

        train_dataset, test_dataset = random_split(dataset,[n_train,n_test])

        self.train_dataset = DataLoader(train_dataset,batch_size=self.batch_size, shuffle=True)
        self.test_dataset = DataLoader(test_dataset,batch_size=self.batch_size,shuffle=True)

    def train_dataloader(self):
        return self.train_dataset
    
    def test_dataloader(self):
        return self.test_dataset

In [8]:
class ConvulationalNetwork(pl.LightningModule):
    def __init__(self):
        super(ConvulationalNetwork,self).__init__()
        self.conv1 = nn.Conv2d(3,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.fc1 = nn.Linear(16*54*54,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, len(class_names))
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16 * 54 * 54)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        return F.log_softmax(X, dim=1)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("test_loss", loss)
        self.log("test_acc", acc)

In [10]:
torch.cuda.is_available()

False

In [9]:
if __name__ == '__main__':
    datamodule = DataModule()
    datamodule.setup()
    model = ConvulationalNetwork()
    trainer = pl.Trainer(max_epochs=20)
    trainer.fit(model = model, datamodule= datamodule)
    datamodule.setup(stage='test')
    test_loader = datamodule.test_dataloader()
    trainer.test(dataloaders=test_loader)

c:\Users\Alvin\Downloads\PythonProjects\RM-Project\.venv\Lib\site-packages\torch\cuda\__init__.py:1059: UserWarning: cudaGetDeviceCount() returned cudaErrorNotSupported, likely using older driver or on CPU machine (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10\cuda\CUDAFunctions.cpp:88.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
c:\Users\Alvin\Downloads\PythonProjects\RM-Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. Fo

Epoch 0:  16%|█▌        | 305/1891 [02:03<10:42,  2.47it/s, v_num=2]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\Alvin\Downloads\PythonProjects\RM-Project\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
